In [5]:
import cv2
import mediapipe as mp
import time

import numpy as np
import time
import torch
import gc
import os
import math

In [6]:
# Создаем класс упрощенный handDetector 
class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        # Конструктор класса
        # mode: Установите значение True, чтобы обнаруживать руки в видеопотоке в реальном времени.
        # maxHands: Максимальное количество рук для обнаружения.
        # detectionCon: Пороговое значение для обнаружения руки. Увеличение значения увеличивает точность, но уменьшает скорость обнаружения.
        # trackCon: Пороговое значение для отслеживания руки. Увеличение значения увеличивает точность отслеживания, но может привести к потере отслеживания в некоторых случаях.

        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        # Импорт необходимых модулей для обнаружения рук
        self.mpHands = mp.solutions.hands
        # Инициализация детектора рук
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img, draw=True):
        # Метод для обнаружения рук на изображении
        # img: Входное изображение
        # draw: Установите значение True, чтобы отрисовать найденные руки на изображении

        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Обработка изображения для обнаружения рук
        self.results = self.hands.process(imgRGB)

        # Если руки обнаружены
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    # Отрисовка обнаруженных рук на изображении
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)

        return img

    def findPosition(self, img, handNo=0, draw=True):
        # Метод для определения позиции ключевых точек руки
        # img: Входное изображение
        # handNo: Индекс руки для определения позиции, если обнаружены несколько рук.
        # draw: Установите значение True, чтобы отрисовать ключевые точки на изображении.

        lmList = []

        # Если руки обнаружены
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # Получение координат ключевых точек в пикселях
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)

                # Сохранение координат ключевых точек в список
                lmList.append([id, cx, cy])

                if draw:
                    # Отрисовка ключевых точек на изображении
                    cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)

        return lmList


In [11]:
# смотрим как работает алгоритм определение руки на изображении
pTime = 0
cTime = 0

# Инициализация видеозахвата с камеры
cap = cv2.VideoCapture(0)

# Создание экземпляра класса handDetector для обнаружения руки
detector = handDetector(mode=True, maxHands=1, detectionCon=1, trackCon=0.25)

while True:
    # Чтение кадра из видеопотока
    success, img = cap.read()

    # Обнаружение руки на кадре
    img = detector.findHands(img, draw=True)

    # Определение позиции ключевых точек руки
    lmList = detector.findPosition(img, draw=False)

    # Расчет FPS (количество кадров в секунду)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    # Вывод значения FPS на изображении
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

    # Отображение изображения с обнаруженной рукой и значением FPS
    cv2.imshow("Image", cv2.resize(img, (640, 480)))

    # Завершение цикла при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break


KeyboardInterrupt: 

In [8]:
# Создаем класс для реализации приложения рисования
class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        """
        Конструктор класса handDetector.
        
        Аргументы:
            mode (bool): Определяет режим обнаружения рук. True - для обнаружения всех рук, False - только для одной руки.
            maxHands (int): Максимальное количество рук для обнаружения.
            detectionCon (float): Уровень доверия для обнаружения руки.
            trackCon (float): Уровень доверия для отслеживания руки.
        """
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

    def findHands(self, img, draw=True):
        """
        Обнаруживает руки на изображении и рисует на них ключевые точки и линии.

        Аргументы:
            img (numpy.ndarray): Исходное изображение.
            draw (bool): Определяет, нужно ли рисовать ключевые точки и линии на изображении.

        Возвращает:
            img (numpy.ndarray): Изображение с нарисованными ключевыми точками и линиями, если draw=True.
        """
        # Конвертируем изображение в RGB
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Обрабатываем изображение с использованием модели руки
        self.results = self.hands.process(imgRGB)
    
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    # Рисуем ключевые точки и линии на изображении
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
    
        return img

    def findPosition(self, img, handNo=0, draw=True):
        """
        Определяет позицию ключевых точек руки.

        Аргументы:
            img (numpy.ndarray): Исходное изображение.
            handNo (int): Индекс руки, для которой нужно определить позицию ключевых точек.
            draw (bool): Определяет, нужно ли рисовать ключевые точки на изображении.

        Возвращает:
            lmList (list): Список с позициями ключевых точек руки. Каждая точка представлена в формате [id, x, y].
        """
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
    
            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax
    
        if draw:
            cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20), (0, 255, 0), 2)
    
        return self.lmList, bbox

    def fingersUp(self):
        """
        Определяет, какие пальцы руки подняты.

        Возвращает:
            fingers (list): Список, где 1 означает поднятый палец, а 0 - опущенный.
        """
        fingers = []
        # Палец-большой
        if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)
    
        # Остальные пальцы
        for id in range(1, 5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
    
        return fingers


In [15]:
# Устанавливаем толщину кисти и ластика
brushThickness = 25
eraserThickness = 100

# Указываем путь к папке с картинками для шапки (не используется в коде)
folderPath = r'D:\1T\task_3\3_3\image'
myList = os.listdir(folderPath)
overlayList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    overlayList.append(image)
header = overlayList[0]
drawColor = (255, 0, 255)

# Инициализируем видеопоток с камеры
cap = cv2.VideoCapture(0)
cap.set(3, 1080)
cap.set(4, 720)

# Создаем экземпляр класса handDetector для обнаружения рук
detector = handDetector(detectionCon=1,maxHands=1)
xp, yp = 0, 0
# Создаем "холст" для рисования
imgCanvas = np.zeros((720, 1080, 3), np.uint8)

while True:
    # Получаем кадр из видеопотока
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img = cv2.resize(img, (1080, 720))
    
    # Находим ключевые точки рук на изображении
    img = detector.findHands(img)
    lmList, _ = detector.findPosition(img, draw=False)
    
    if len(lmList) != 0:
        # Определяем позиции ключевых точек указательного и среднего пальцев
        x1, y1 = lmList[8][1:]
        x2, y2 = lmList[12][1:]
    
        # Проверяем, какие пальцы подняты
        fingers = detector.fingersUp()
        #print(fingers)
        
        # Если подняты указательный и средний пальцы, находим нужный цвет
        if fingers[1] and fingers[2]:
            xp, yp = 0, 0
           # print('Selection Mode')
            # Проверяем, в какой области экрана находится палец для выбора цвета
            if y1 < 115:
                if 160 < x1 < 233:  # Красный
                    header = overlayList[0]
                    drawColor =  (49, 49, 255)
                elif 270 < x1 < 340:  # Оранжевый
                    header = overlayList[1]
                    drawColor = (77, 145, 255)
                elif 370 < x1 < 445: # Желтый
                    header = overlayList[2]
                    drawColor = (89, 222, 255)
                elif 470 < x1 < 550:
                    header = overlayList[3]
                    drawColor = (87, 217, 126)
                elif 570 < x1 < 650:
                    header = overlayList[4]
                    drawColor = (255, 182, 56)
                elif 670 < x1 < 750:
                    header = overlayList[5]
                    drawColor = (255, 113, 82) 
                elif 770 < x1 < 850:
                    header = overlayList[6]
                    drawColor = (255, 82, 140)  
                elif 890 < x1 < 1080: #
                    header = overlayList[7]
                    drawColor = (0, 0, 0)
                                 
                cv2.rectangle(img, (x1, y1 - 25), (x2, y2 + 25), drawColor, cv2.FILLED)
            
                
        # 5. If Drawing Mode – Index finger is up
        if fingers[1] and fingers[2] == False:
            cv2.circle(img, (x1, y1), 15, drawColor, cv2.FILLED)
            #print('Drawing Mode')
            if xp == 0 and yp == 0:
                xp, yp = x1, y1
        
            cv2.line(img, (xp, yp), (x1, y1), drawColor, brushThickness)
        
            if drawColor == (0, 0, 0):
                cv2.line(img, (xp, yp), (x1, y1), drawColor, eraserThickness)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, eraserThickness)
        
            else:
                cv2.line(img, (xp, yp), (x1, y1), drawColor, brushThickness)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, brushThickness)
        
            xp, yp = x1, y1
        
        # # Clear Canvas when all fingers are up
        if all (x >= 1 for x in fingers):
             imgCanvas = np.zeros((720, 1080, 3), np.uint8)

        
    imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
    _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)
    imgInv = cv2.cvtColor(imgInv, cv2.COLOR_GRAY2BGR)

    # Resize imgInv to match the size of img
    #imgInv = cv2.resize(imgInv, (img.shape[1], img.shape[0]))
    #imgCanvas = cv2.resize(imgCanvas, (img.shape[1], img.shape[0]))

    img = cv2.bitwise_and(img, imgInv)
    img = cv2.bitwise_or(img, imgCanvas)
    
    # Setting the header image
    img[0:110, 0:1080] = header
    img = cv2.addWeighted(img,0.8,imgCanvas,0.5,0)
    cv2.imshow('Image', img)
    #cv2.imshow('Canvas', imgCanvas)
    #cv2.imshow('Inv', imgInv)
    if cv2.waitKey(1):
        if 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
        if len(lmList) != 0:
            if (fingers[4] and fingers[3]==False and fingers[2]==False and fingers[1]==False):
                cap.release()
                cv2.destroyAllWindows()
                break